In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
pip install --ignore-installed --upgrade tensorflow

     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 26.1MB 115kB/s 
     |████████████████████████████████| 51kB 163kB/s 
     |████████████████████████████████| 20.2MB 158kB/s 
     |████████████████████████████████| 3.8MB 35.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 2.7MB 34.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 450kB 43.7MB/s 
     |████████████████████████████████| 112kB 48.8MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 1.3MB 36.9MB/s 
     |████████████████████████████████| 593kB 49.1MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 307kB 48.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 2.9MB 40.5MB/s 
    

In [0]:

import os
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import warnings
warnings.filterwarnings('ignore')
from skimage.segmentation import clear_border
import keras


In [0]:

jimread = lambda x: np.expand_dims(imread(x)[::4, ::4],0)

In [0]:
lungs_images = glob(os.path.join('/content/drive/My Drive/2d_images', '*.tif'))
lungs_masks = ['_masks'.join(c_file.split('_images')) for c_file in lungs_images]

In [10]:
images = np.stack([jimread(i) for i in lungs_images], 0)
masks = np.stack([jimread(i) for i in lungs_masks], 0) / 255.0

#Preprocess
preImages = np.stack([imread(i)[::4, ::4] for i in lungs_images], 0)
for idx,image in enumerate(preImages):
    binary = image < -400
    cleared = clear_border(binary)
    preImages[idx] = preImages[idx] * cleared

#plt.imshow(preImages[0])
images = np.expand_dims(preImages, 1)
print(images.shape)

(267, 1, 128, 128)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(images, masks, test_size=0.1)
print('Training input is', X_train.shape)
print('Training output is {}, min is {}, max is {}'.format(y_train.shape, y_train.min(), y_train.max()))
print('Testing set is', X_test.shape)

Training input is (240, 1, 128, 128)
Training output is (240, 1, 128, 128), min is 0.0, max is 1.0
Testing set is (27, 1, 128, 128)


In [0]:
SEED=42
#Data augmentation
def image_augmentation_generator(xtrain, ytrain, batch_size):
    data_generator = ImageDataGenerator(
            rotation_range=90).flow(xtrain, xtrain, batch_size, seed=SEED)
    mask_generator = ImageDataGenerator(
           rotation_range=90).flow(ytrain, ytrain, batch_size, seed=SEED)
    while True:
        x_batch, _ = data_generator.next()
        y_batch, _ = mask_generator.next()
        yield x_batch, y_batch

In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

In [0]:
def buildmodel(hp):
  model = keras.Sequential([
             keras.layers.Conv2D(        
                      filters = hp.Int('conv1',min_value = 32,max_value = 128,step = 16),
                      kernel_size = (3,3),
                      activation = 'sigmoid',
                      input_shape = images.shape[1:],
                      padding = 'same'
             ),
             keras.layers.Conv2D(
                       filters = hp.Int('conv2',min_value = 32,max_value = 128,step = 16),
                       kernel_size = (3,3),
                       activation = 'sigmoid',
                       input_shape = images.shape[1:],
                      padding = 'same'
            ),
             keras.layers.MaxPool2D(
                 pool_size = (2,2),padding = 'same'
             ),
             keras.layers.Dense(64,activation = 'sigmoid'),
             keras.layers.Conv2D(filters = hp.Int('k',min_value = 1,max_value = 128,step = 16),
             kernel_size = (3,3),
             activation = 'sigmoid',
             input_shape = image.shape[1:],
             padding = 'same' 
             ),
             keras.layers.UpSampling2D(size=(2,2))
  ])
  model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=[dice_coef])
  return model

In [0]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=3b260c6138f396a8d4fe4f5da69d930f6675b429241bf333ef717daffea1fe82
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=81bdc37c7d4ab03258cf37582e9fba439509cded1744a3039a0eb31738e93387
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
from kerastuner.tuners import RandomSearch

In [0]:
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(buildmodel,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="lungs nodules detection")

RuntimeError: ignored

In [0]:
import keras.backend as K